In [ ]:
# !pip install -U transformers datasets pyyaml torch torchvision torchaudio sentencepiece torch torchvision torchaudio sacremoses

In [60]:
# !python -m spacy download en_core_web_md

In [61]:
from transformers import pipeline
import yaml, json, sys
from yaml.loader import SafeLoader

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
translator("Hi there. How was your day today?")

In [62]:
def translator_pipe(language="SPA"):
    if language == "SPA":
        return pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
    elif language == "POR":
        return pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
    else:
        print("Sorry, this language is not supported. Please try again and use SPA for Spanish or POR for portuguese.")

In [5]:
def open_workshop(path):
    with open(path) as f:
        return yaml.load(f, Loader=SafeLoader)

In [4]:
path = "m-mastercard-crypto.yml"

In [8]:
nlp = spacy.load("en_core_web_md")

sample = nlp(data["slides"][25]["notes"])

for num, sentence in enumerate(sample.sents):
    print(f"Sentence #{num}:\nIn English:\n{sentence.text}\nIn Spanish:\n{translator(sentence.text)[0]['translation_text']}\n")

In [57]:
from tqdm import tqdm

def extractor(data):
    full_notes = []
    for idx, slide in enumerate(tqdm(data["slides"])):
        if "notes" in slide.keys():
            one_note = slide["notes"].split("\n")
            for num, sentence in enumerate(one_note):
                if sentence != None and len(sentence) > 3:
                    if len(sentence) < 400:
                        one_note[num] = translator(sentence)[0]['translation_text']
                    elif len(sentence) > 400:
                        group_sent = sentence.split(".")
                        for num2, sent2 in enumerate(group_sent):
                            group_sent[num2] = translator(sent2)[0]['translation_text']
                        group_sent2 = ".".join(sent for sent in group_sent)
                        one_note[num] = group_sent2
                else:
                    one_note[num] = sentence
            slide["notes"] = "\n".join(sent for sent in one_note)
        if "text" in slide.keys():
            text = slide["text"]
            slide["text"] = translator(text)[0]['translation_text']
        if "title" in slide.keys():
            title = slide["title"]
            slide["title"] = translator(title)[0]['translation_text']
        data["slides"][idx] = slide
    return data

In [ ]:
%%time

test = extractor(data)

In [ ]:
%%time

for idx, slide in enumerate(data["slides"]):
    full_notes = []
    if "notes" in slide.keys():
        one_note = slide["notes"].split("\n")
        for num, sentence in enumerate(one_note):
            if sentence != None:
                one_note[num] = translator(sentence)[0]['translation_text']
            # full_notes.append()
        # data['slides'][idx]["notes"] = " ".join(nlp(sent).text for sent in full_notes)
        slide["notes"] = "\n".join(sent for sent in one_note)
        # pprint(slide["notes"])
    if "text" in slide.keys():
        text = slide["text"]
        slide["text"] = translator(text)[0]['translation_text']
        # print(slide["text"])
    if "title" in slide.keys():
        title = slide["title"]
        slide["title"] = translator(title)[0]['translation_text']
    #     print(slide["title"])
    # if idx > 5:
    #     break
    # try:
    #   data["slides"][idx]["notes"] = translator(data["slides"][idx]["notes"])[0]["translation_text"]
    # except:
    #   print(f"An error occured at index {idx}.")
print("Done ✅")

('# Configuración y preparación Toda la siguiente información se puede '
 'encontrar en la [Base de datos '
 'Coda](https://coda.io/d/Global-Mindset-Dashboard_dkQbmjYn9I/Content-Database_su9Cd#Content-Database_tu_Lj/r1) '
 '(si es facilitador interno) o en tu PDF de Formación de Contenidos '
 'ambientales por correo electrónico a ti (si es facilitador externo). - Deck '
 'abierto, Menti y Toby - Lista abierta (asegurado de que tú y tu '
 'cofacilitador está de acuerdo con quién está permitido) - Revisa el sonido '
 'para facilitar y facilitar el acceso al sitio web (que tú y tu cofacilitador '
 'está de acuerdo con lo que está permitido) - Revisar el sonido para '
 'facilitar y facilitar el acceso a la web, y facilitar el acceso a la web.\n'
 "- `?client={default, citi &rarr; 'Mastercard', 'Citibank' - setting "
 "client='citi' permite a los facilitadores elegir cual referencia de "
 'Mastercard/reemplazar con Citibank cuando corresponda.\n'
 '- Ver documentación sobre el uso de [etiqu

In [54]:
data["slides"][30]["notes"].split("\n")

['# Punto',
 'Los números de página identifican las páginas y también las mantienen en orden.',
 '',
 '## Narrativa',
 '**Tell:**',
 '- Así que los números de página funcionan muy bien si tenemos un libro, pero no puedes numerar páginas de datos digitales.',
 'trozos, o bloques, de datos digitales y asegúrese de que se mantienen en orden?',
 '',
 '### Transición',
 'Para los datos digitales, utilizamos el concepto llamado hashing.',
 '',
 '[NS: Hashing]',
 '']

In [48]:
data.keys()

dict_keys(['name', 'tags', 'description-text', 'contributors', 'tag-parameters', 'slides'])

In [47]:
data

{'name': 'Mastercard Digital in a Day - Crypto 101',
 'tags': ['client-mastercard',
  'client-citibank',
  'bespoke',
  'mindset',
  'crypto',
  'blockchain',
  'defi'],
 'description-text': 'A three-hour workshop as an extension to Digital in a Day 1.0, also adjusted for Citibank.',
 'contributors': ['Marc Jansen', 'Diksha Garg'],
 'tag-parameters': {'audience': {'default': '',
   'diad1': 'Welcome back after Digital in a Day 1.0',
   'knt': 'Knowledge and Trends'},
  'client': {'default': 'Mastercard', 'citi': 'Citibank'}},
 'slides': [{'type': 'decoded logo',
   'notes': " # Configuración y preparación Toda la siguiente información se puede encontrar en la [Base de datos Coda](https://coda.io/d/Global-Mindset-Dashboard_dkQbmjYn9I/Content-Database_su9Cd#Content-Database_tu_Lj/r1) (si eres facilitador interno) o en tu PDF de Formación de Contenidos enviado por correo electrónico a ti (si eres facilitador externo). - Deck abierto, Menti y Toby - Lista abierta (asegurate de que tú y tu 

In [56]:
def dave_it(workshop, path):
    with open(path, 'w') as file:
        yaml.dump(workshop, file)